In [0]:
# This example demonstrates how use neural networks for analyzing real estate data
# The following Databricks tutorial contains more information: 
# https://docs.databricks.com/_static/notebooks/getting-started/get-started-keras-dbr7ml.html

# Setup
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
import mlflow
import mlflow.keras
import mlflow.tensorflow

In [0]:
# Load and preprocess data
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_housing = fetch_california_housing()

In [0]:
# inspect the data
# Note: the data is not a DataFrame (yet)
cal_housing

Out[7]: {'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_da

In [0]:
# Split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                    cal_housing.target,
                                                    test_size=0.2)

In [0]:
# Inspect data
X_train

Out[11]: array([[   2.1265    ,   32.        ,    4.43631436, ...,    2.77777778,
          37.98      , -121.26      ],
       [   5.9359    ,   16.        ,    5.7560241 , ...,    3.48795181,
          37.26      , -121.81      ],
       [   5.6732    ,   43.        ,    5.90709046, ...,    2.51344743,
          33.96      , -118.4       ],
       ...,
       [   2.6962    ,   36.        ,    4.09699769, ...,    1.90300231,
          32.74      , -117.25      ],
       [   0.9797    ,   32.        ,    4.84328358, ...,    2.67164179,
          37.95      , -122.37      ],
       [   2.0962    ,   28.        ,    6.41324201, ...,    2.49543379,
          40.28      , -123.84      ]])

In [0]:
X_test

Out[14]: array([[   1.6786    ,   52.        ,    5.42599278, ...,    2.        ,
          35.37      , -119.03      ],
       [   5.0142    ,   15.        ,    5.98195489, ...,    3.63609023,
          33.99      , -117.95      ],
       [   2.2765    ,   38.        ,    5.6031746 , ...,    3.76643991,
          34.16      , -118.15      ],
       ...,
       [   7.9234    ,   29.        ,    8.01769912, ...,    2.89675516,
          37.26      , -121.99      ],
       [   3.7813    ,   36.        ,    6.63461538, ...,    3.03846154,
          33.84      , -118.1       ],
       [   3.2011    ,   17.        ,    3.52339181, ...,    2.15204678,
          33.93      , -118.11      ]])

In [0]:
y_train

Out[15]: array([0.557, 2.495, 3.307, ..., 2.882, 0.764, 0.74 ])

In [0]:
y_test

Out[16]: array([1.266  , 2.699  , 1.466  , ..., 5.00001, 2.091  , 1.625  ])

In [0]:
# Scale features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
# create neural network
def create_model():
  model = Sequential()
  model.add(Dense(20, input_dim=8, activation="relu"))
  model.add(Dense(20, activation="relu"))
  model.add(Dense(1, activation="linear"))
  return model

In [0]:
# compile the model
model = create_model()
model.compile(loss="mse",
              optimizer="Adam",
              metrics=["mse"])

In [0]:
# Create callbacks

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

experiment_log_dir = "/tmp/tb"
checkpoint_path = "/tmp/keras_checkpoint_weights.ckpt"

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=experiment_log_dir)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor="loss", mode="min", patience=3)

history = model.fit(X_train, y_train, validation_split=.2, epochs=35, callbacks=[tensorboard_callback, model_checkpoint, early_stopping])


Epoch 1/35
 1/413 [..............................] - ETA: 3:08 - loss: 8.6864 - mse: 8.6864 25/413 [>.............................] - ETA: 0s - loss: 6.9109 - mse: 6.9109  53/413 [==>...........................] - ETA: 0s - loss: 6.0994 - mse: 6.0994 91/413 [=====>........................] - ETA: 0s - loss: 4.9167 - mse: 4.9167125/413 [========>.....................] - ETA: 0s - loss: 4.1482 - mse: 4.1482159/413 [==========>...................] - ETA: 0s - loss: 3.5088 - mse: 3.5088193/413 [=============>................] - ETA: 0s - loss: 3.2191 - mse: 3.2191227/413 [===============>..............] - ETA: 0s - loss: 2.8649 - mse: 2.8649267/413 [==================>...........] - ETA: 0s - loss: 2.5686 - mse: 2.5686308/413 [=====================>........] - ETA: 0s - loss: 2.3295 - mse: 2.3295354/413 [========================>.....] - ETA: 0s - loss: 2.1200 - mse: 2.1200399/413 [===========================>..] - ETA: 0s - loss: 1.9590 - mse: 1.9590
Epoch 00001: val_loss improved from inf to 0.69122, saving model to /dbfs/<username>/keras_checkpoint_weights.ckpt
INFO:tensorflow:Assets written to: /dbfs/<username>/keras_checkpoint_weights.ckpt/assets
413/413 [==============================] - 3s 6ms/step - loss: 1.9131 - mse: 1.9131 - val_loss: 0.6912 - val_mse: 0.6912
Epoch 2/35
 1/413 [..............................] - ETA: 1s - loss: 1.2608 - mse: 1.2608 46/413 [==>...........................] - ETA: 0s - loss: 0.6402 - mse: 0.6402 88/413 [=====>........................] - ETA: 0s - loss: 0.6120 - mse: 0.6120131/413 [========>.....................] - ETA: 0s - loss: 0.6162 - mse: 0.6162175/413 [===========>..................] - ETA: 0s - loss: 0.6093 - mse: 0.6093219/413 [==============>...............] - ETA: 0s - loss: 0.6463 - mse: 0.6463261/413 [=================>............] - ETA: 0s - loss: 0.6253 - mse: 0.6253307/413 [=====================>........] - ETA: 0s - loss: 0.6112 - mse: 0.6112352/413 [========================>.....] - ETA: 0s - loss: 0.5954 - mse: 0.5954393/413 [===========================>..] - ETA: 0s - loss: 0.5881 - mse: 0.5881
Epoch 00002: val_loss improved from 0.69122 to 0.47456, saving model to /dbfs/<username>/keras_checkpoint_weights.ckpt
INFO:tensorflow:Assets written to: /dbfs/<username>/keras_checkpoint_weights.ckpt/assets
